In [7]:
import requests,json
from IPython.display import display
from bis import bis
from bis import itis
from bis import tir
from bis2 import couchconnect
import time
import couchdb

In [8]:
couch = couchdb.Server()

In [9]:
dbSource = couch["tir-gapspecies"]
dbTarget = couch["tir-itis"]

In [16]:
for id in dbSource:
    gapSpecies = dbSource[id]
    if "ITIS_TSN" in gapSpecies and gapSpecies["ITIS_TSN"] not in dbTarget:
        itisSearchURL = itis.getITISSearchURL(gapSpecies["ITIS_TSN"],False,False)
    else:
        if "ITIS_TSN" not in gapSpecies:
            itisSearchURL = itis.getITISSearchURL(gapSpecies["scientificname"],False,False)

    itisSearchResults = requests.get(itisSearchURL).json()
    if len(itisSearchResults["response"]["docs"]) == 1:
        itisDoc = itisSearchResults["response"]["docs"][0]
        itisData = itis.packageITISJSON("TSN Query",gapSpecies["ITIS_TSN"],itisDoc)
        display (itisData)
        dbTarget[gapSpecies["ITIS_TSN"]] = itisData
    else:
        print (itisSearchURL)
        display (itisSearchResults)
        break
            

http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:Sceloporus\%20uniformis


{'response': {'docs': [], 'numFound': 0, 'start': 0},
 'responseHeader': {'QTime': 0,
  'params': {'q': 'nameWOInd:Sceloporus\\ uniformis',
   'rows': '10',
   'wt': 'json'},
  'status': 0}}

In [3]:
# Set up the actions/targets for this particular instance
thisRun = {}
thisRun["instance"] = "DataDistillery"
thisRun["db"] = "BCB"
thisRun["baseURL"] = gc2.sqlAPI(thisRun["instance"],thisRun["db"])
thisRun["commitToDB"] = True
thisRun["totalRecordsToProcess"] = 5
thisRun["totalRecordsProcessed"] = 0

numberWithoutTIRData = 1

while numberWithoutTIRData == 1 and thisRun["totalRecordsProcessed"] < thisRun["totalRecordsToProcess"]:

    q_recordToSearch = "SELECT id, \
        registration->>'source' AS source, \
        registration->>'followTaxonomy' AS followtaxonomy, \
        registration->>'taxonomicLookupProperty' AS taxonomiclookupproperty, \
        registration->>'scientificname' AS scientificname, \
        registration->>'tsn' AS tsn \
        FROM tir.tir \
        WHERE itis IS NULL \
        LIMIT 1"
    recordToSearch  = requests.get(thisRun["baseURL"]+"&q="+q_recordToSearch).json()
    
    numberWithoutTIRData = len(recordToSearch["features"])
    
    if numberWithoutTIRData == 1:
        tirRecord = recordToSearch["features"][0]
        
        # Set up a local data structure for storage and processing
        thisRecord = {}

        # Set data from query results
        thisRecord["id"] = tirRecord["properties"]["id"]
        thisRecord["source"] = tirRecord["properties"]["source"]
        thisRecord["followTaxonomy"] = tirRecord["properties"]["followtaxonomy"]
        thisRecord["taxonomicLookupProperty"] = tirRecord["properties"]["taxonomiclookupproperty"]
        thisRecord["tsn"] = tirRecord["properties"]["tsn"]
        thisRecord["scientificname"] = tirRecord["properties"]["scientificname"]
        thisRecord["scientificname_search"] = bis.cleanScientificName(thisRecord["scientificname"])

        # Set defaults for thisRecord
        thisRecord["matchMethod"] = "Not Matched"
        thisRecord["matchString"] = thisRecord["scientificname_search"]
        thisRecord["itisData"] = itis.packageITISJSON(thisRecord["matchMethod"],thisRecord["matchString"],0)
        thisRecord["numResults"] = 0
        itisDoc = {}

        if thisRecord["taxonomicLookupProperty"] == "scientificname" and len(thisRecord["scientificname_search"]) != 0:

            thisRecord["itisSearchURL"] = itis.getITISSearchURL(thisRecord["scientificname_search"],False,True)

            # Try an exact match search
            try:
                itisSearchResults = requests.get(thisRecord["itisSearchURL"]).json()
                thisRecord["numResults"] = len(itisSearchResults["response"]["docs"])
            except Exception as e:
                print (e)
                pass


            # If we got only a single record on an exact match search, set the method and proceed
            if thisRecord["numResults"] == 1:
                thisRecord["matchMethod"] = "Exact Match"
                itisDoc = itisSearchResults["response"]["docs"][0]

            # If we found nothing on an exact match search, try a fuzzy match
            elif thisRecord["numResults"] == 0:
                try:
                    thisRecord["itisSearchURL"] = itis.getITISSearchURL(thisRecord["scientificname_search"],True,True)
                    itisSearchResults = requests.get(thisRecord["itisSearchURL"]).json()
                    thisRecord["numResults"] = len(itisSearchResults["response"]["docs"])
                except Exception as e:
                    print (e)
                    pass
                if thisRecord["numResults"] == 1:
                    thisRecord["matchMethod"] = "Fuzzy Match"
                    itisDoc = itisSearchResults["response"]["docs"][0]

            # If we got a result but the usage is not accepted/invalid and we should follow taxonomy for this record, then retrieve the record for the accepted TSN
            if len(itisDoc) > 0 and itisDoc["usage"] in ["not accepted","invalid"] and thisRecord["followTaxonomy"]:
                thisRecord["itisSearchURL"] = itis.getITISSearchURL(itisDoc["acceptedTSN"][0],False,False)
                try:
                    itisSearchResults = requests.get(thisRecord["itisSearchURL"]).json()
                except Exception as e:
                    print (e)
                    pass
                if thisRecord["numResults"] == 1:
                    thisRecord["matchMethod"] = "Followed Accepted TSN"
                    itisDoc = itisSearchResults["response"]["docs"][0]

            # If we got an ITIS Doc returned, package the results
            if len(itisDoc) > 0:
                thisRecord["itisData"] = itis.packageITISJSON(thisRecord["matchMethod"],thisRecord["matchString"],itisDoc)

        elif thisRecord["taxonomicLookupProperty"] == "tsn" and thisRecord["tsn"] is not None:
            thisRecord["itisSearchURL"] = itis.getITISSearchURL(thisRecord["tsn"],False,False)
            itisSearchResults = requests.get(thisRecord["itisSearchURL"]).json()
            thisRecord["matchMethod"] = "TSN Query"
            thisRecord["matchString"] = thisRecord["tsn"]
            itisDoc = itisSearchResults["response"]["docs"][0]
            thisRecord["itisData"] = itis.packageITISJSON(thisRecord["matchMethod"],thisRecord["matchString"],itisDoc)
            
        display (thisRecord)
        if thisRun["commitToDB"]:
            print (tir.cacheToTIR(thisRun["baseURL"],thisRecord["id"],"itis",json.dumps(thisRecord["itisData"])))
        thisRun["totalRecordsProcessed"] = thisRun["totalRecordsProcessed"] + 1

        

{'followTaxonomy': 'true',
 'id': 7382,
 'itisData': {'MatchMethod': 'Fuzzy Match',
  'MatchString': 'Acipenser oxyrhinchus desotoi',
  'cacheDate': '2017-08-14T18:20:23.660013',
  'commonnames': [{'language': 'English', 'name': 'Gulf sturgeon'}],
  'createDate': '2004-02-23 13:21:38',
  'hierarchy': [{'name': 'Animalia', 'rank': 'Kingdom'},
   {'name': 'Bilateria', 'rank': 'Subkingdom'},
   {'name': 'Deuterostomia', 'rank': 'Infrakingdom'},
   {'name': 'Chordata', 'rank': 'Phylum'},
   {'name': 'Vertebrata', 'rank': 'Subphylum'},
   {'name': 'Gnathostomata', 'rank': 'Infraphylum'},
   {'name': 'Actinopterygii', 'rank': 'Superclass'},
   {'name': 'Chondrostei', 'rank': 'Class'},
   {'name': 'Acipenseriformes', 'rank': 'Order'},
   {'name': 'Acipenseroidei', 'rank': 'Suborder'},
   {'name': 'Acipenseridae', 'rank': 'Family'},
   {'name': 'Acipenserinae', 'rank': 'Subfamily'},
   {'name': 'Acipenser', 'rank': 'Genus'},
   {'name': 'Acipenser oxyrinchus', 'rank': 'Species'},
   {'name': '

{'_execution_time': 0.083, 'affected_rows': 1, 'success': True, 'auth_check': {'success': True, 'auth_level': 'Write', 'checked_relations': ['tir.tir'], 'session': None}}
